In [56]:
from bls_py import (aggregation_info
    ,bls
    ,bls12381
    ,ec
    ,fields
    ,fields_t
    ,fields_t_c
   , keys
   , pairing
   , signature
    ,tdata
    ,tests
    ,threshold
    ,util)

from random import randint

We will work in the G1 (Fq) group since public keys are in the G1 group.

In [57]:
G = ec.generator_Fq()
l = ec.default_ec.n

In [58]:
def Hp(point: bls.AffinePoint) -> bls.AffinePoint:
    return ec.hash_to_point_Fq(point.serialize())

def Hs(*xs):
    h = b''
    for x in xs:
        if type(x) == str:
            b = bytes(x, encoding="utf-8")
        elif type(x) == bls.AffinePoint:
            b = x.serialize()
        else:
            b = bytes(x)
        h = ec.hash256(h + b)
    return int.from_bytes(h, "big") % l

In [81]:
N = 21
m = "important message"

xs = [bls.PrivateKey.from_seed(str(i)).value for i in range(N)]
P = [x * G for x in xs]

# j is the index of the secret signer
j = 0
x = xs[0]

In [82]:
# Key Image
I = x * Hp(P[j])

In [83]:
alpha = randint(0, l)
s = [randint(0, l) if i != j else None for i in range(N)]

In [84]:
L = [None] * N
R = [None] * N
c = [None] * N

L[j] = alpha * G
R[j] = alpha * Hp(P[j])
c[(j+1) % N] = Hs(m, L[j], R[j])

for i in range(1, N):
    idx = (j + i) % N
    L[idx] = s[idx] * G + c[idx] * P[idx]
    R[idx] = s[idx] * Hp(P[idx]) + c[idx] * I
    c[(idx + 1) % N] = Hs(m, L[idx], R[idx])

In [85]:
s[j] = (alpha - (c[j] * x)) % l

In [86]:
sig = (I, c[0], s)

In [87]:
alpha == (s[j] + c[j] * x) % l

True

In [88]:
L[j] == alpha * G

True

In [89]:
alpha * G == s[j] * G + c[j] * x * G

True

In [90]:
alpha * G == s[j] * G + (c[j] * x % l) * G

True

In [91]:
c[j] * x  * G == c[j] * P[j]

True

In [92]:
alpha * G == s[j] * G + c[j] * P[j]

True

In [93]:
ec.default_ec.n

52435875175126190479447740508185965837690552500527637822603658699938581184513

In [94]:
s[j]

10911266535558011642090291323489759204648573621235556134517723471565392816376

In [95]:
# Now to verify

def verify(I, c0, s, P, m):
    N = len(P)
    
    Lp = [None] * N
    Rp = [None] * N
    cp = [None] * N
    cp[0] = c0

    for i in range(N):
        Lp[i] = s[i] * G + cp[i] * P[i]
        Rp[i] = s[i] * Hp(P[i]) + cp[i] * I
        cp[(i + 1) % N] = Hs(m, Lp[i], Rp[i])
        
    assert cp[0] == c0
    assert Hs(m, Lp[-1], Rp[-1]) == cp[0]
    print("Everything checks out")

In [96]:
verify(*sig, P, m)

Everything checks out


In [55]:
c

[4743997262915561299125662112332479794546814593664826494615037867389839415761,
 48593580839996011308605559297217465500240338083195273690359642139437199354217,
 6112860136447671528098358327700418968737786762848384794127819209703304836358]